# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need


# 2) instantiate a Spark session 
spark = SparkSession.builder.appName("Spark Data Quiz").getOrCreate()

# 3) read in the data set located at the path "data/sparkify_log_small.json"
spark_df = spark.read.json("data/sparkify_log_small.json")

# 4) create a view to use with your SQL queries
spark_df.createOrReplaceTempView("logs")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [2]:
# Pages that userId = "" visited
spark.sql(
    """
    SELECT DISTINCT page 
    FROM logs
    WHERE userId == ""
    ORDER BY page asc
    """).show()

+-----+
| page|
+-----+
|About|
| Help|
| Home|
|Login|
+-----+



In [3]:
# All pages
spark.sql(
    """
    SELECT DISTINCT page 
    FROM logs
    ORDER BY page asc
    """).show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [8]:
# All pages EXCEPT the pages that userId = "" visited 
unseen_pages = spark.sql(
    """
    SELECT DISTINCT page 
    FROM  logs
    WHERE page NOT IN
          (
           SELECT DISTINCT page 
           FROM logs 
           WHERE userId == "")
    ORDER BY page asc
    """)
pages_df = unseen_pages.toPandas()
print(pages_df)

               page
0         Downgrade
1             Error
2            Logout
3          NextSong
4     Save Settings
5          Settings
6  Submit Downgrade
7    Submit Upgrade
8           Upgrade


# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

SQL is great for queries. It was a lot easier to get to the right not seen pages via a sub-query than by substracting results using python functions. 

It was easier to learn SQL than figuring out the best python function for a specific use-case.

SQL might be a bit more verbose for simple queries, where a simple python function suffices. 

# Question 3

How many female users do we have in the data set?

In [36]:
females = spark.sql("""
                        SELECT DISTINCT userId 
                        FROM logs 
                        WHERE gender = 'F'
                    """).count()
females

462

# Question 4

How many songs were played from the most played artist?

In [81]:
rockstar = spark.sql("""
                        SELECT artist, COUNT(*) as plays
                        FROM logs
                        WHERE page = "NextSong"
                        GROUP BY artist
                        ORDER BY plays desc
                        LIMIT 1
                     """)
df = rockstar.toPandas()
df

,artist,plays
0,Coldplay,83


# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [17]:
# create a function that identifies Next Songs
spark.udf.register("is_ns", lambda x: 1 if(x == 'NextSong') else 0)

# create a function that identifies Next Songs
spark.udf.register("is_home", lambda x: 1 if(x == 'Home') else 0)

# create a function that counts Next Songs per User
spark.udf.register("ns_per_", lambda x: )


<function __main__.<lambda>(x)>

In [30]:
avg_songs_btw_home = spark.sql("""
                                   SELECT  userId, page, nextS, home, 
                                           SUM(home) OVER(PARTITION BY userId) AS sum_home
                                   FROM (
                                           SELECT userId, page, is_ns(page) as nextS, is_home(page) as home                                           
                                           FROM logs
                                           WHERE userId != '' AND page IN ('NextSong', 'Home')
                                           ORDER BY userId, ts asc
                                        )
                                
                                   
                                   ORDER BY userId
                                   LIMIT 20
                                
                               """)

df = avg_songs_btw_home.toPandas()
df

,userId,page,nextS,home,sum_home
0,10,NextSong,1,0,0.0
1,10,NextSong,1,0,0.0
2,100,NextSong,1,0,3.0
3,100,NextSong,1,0,3.0
4,100,Home,0,1,3.0
5,100,NextSong,1,0,3.0
6,100,Home,0,1,3.0
7,100,Home,0,1,3.0
8,100,NextSong,1,0,3.0
9,100,NextSong,1,0,3.0


In [19]:
# for user in userId 

#   counter = 0
#   has been to home = 0

#      for log in ordered logs: 
#            if home 
#               has been = 1
